<a href="https://colab.research.google.com/github/NSR9/Extensive-Vision-AI/blob/main/Assignment_4/Model_training/Session_4_assign_Nb_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Write MNIST code such that:

* 99.4% validation accuracy 
* Less than 20k Parameters
* You can use anything from above you want.
* Less than 20 Epochs

###Things used: - 
* Activation Function as **ReLU** is used after conv layers
* **MaxPool Layer of 2 x 2** is used twice in the network. 
* Added **batch normalization** after every conv layer
* Added **dropout of 0.069** after each conv layer
* Added **Global average pooling** before the FC layer and then added the FC to get output classes.
* Use learning rate of 0.02 and momentum 0.8

####Parameters count reduced to as low as **14906**

#### Achieved an highest accuracy of **99.49%** at the **16th Epoch**, Where the model got its first accuracy more than **99.40%** at it 9th Epoch only. 


In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3) #28, 26
        self.conv1_bn = nn.BatchNorm2d(16) 
        self.dropout1 = nn.Dropout2d(0.069)

        self.conv2 = nn.Conv2d(16, 16, 3) # 26, 24
        self.conv2_bn = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout2d(0.069)

        self.pool1 = nn.MaxPool2d(2, 2) #24, 12

        self.conv3 = nn.Conv2d(16, 32, 3) #12, 10
        self.conv3_bn = nn.BatchNorm2d(32)
        self.dropout3 = nn.Dropout2d(0.069)

        self.conv4 = nn.Conv2d(32, 16, 3) #10, 8
        self.conv4_bn = nn.BatchNorm2d(16)
        self.dropout4 = nn.Dropout2d(0.069)

        self.pool2 = nn.MaxPool2d(2, 2) #8, 4

        self.conv5 = nn.Conv2d(16, 16, 3) #4 ,2
        self.conv5_bn = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout2d(0.069)

        self.fc = nn.Linear(16 * 2 * 2, 10)

    def forward(self, x):
        x = self.dropout1(self.conv1_bn(F.relu(self.conv1(x))))
        x = self.pool1(self.dropout2(self.conv2_bn(F.relu(self.conv2(x)))))

        x = self.dropout3(self.conv3_bn(F.relu(self.conv3(x))))
        x = self.pool2(self.dropout4(self.conv4_bn(F.relu(self.conv4(x)))))
        
        x = self.dropout5(self.conv5_bn(self.conv5(x)))
        x = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.log_softmax(x)

In [ ]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]           4,640
       BatchNorm2d-9           [-1, 32, 10, 10]              64
        Dropout2d-10           [-1, 32, 10, 10]               0
           Conv2d-11             [-1, 16, 8, 8]           4,624
      BatchNorm2d-12             [-1, 16, 8, 8]              32
        Dropout2d-13             [-1, 16, 8, 8]               0
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item():.10f} batch_id={batch_idx:05d}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.8)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0508, Accuracy: 9880/10000 (98.80%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0322, Accuracy: 9904/10000 (99.04%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0182, Accuracy: 9940/10000 (99.40%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0168, Accuracy: 9942/10000 (99.42%)



epoch=20 loss=0.0800340548 batch_id=00468: 100%|██████████| 469/469 [00:11<00:00, 42.27it/s]



Test set: Average loss: 0.0181, Accuracy: 9943/10000 (99.43%)

